In [1]:
import numpy as np # linear algebra
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading JSON file and Preprocessing

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/convolve-epoch1/sample_submission.csv
/kaggle/input/convolve-epoch1/train.json
/kaggle/input/convolve-epoch1/test.csv


In [ ]:
import json
json_file_path = "/content/drive/MyDrive/convolve/train_new.json"
f = open(json_file_path,)
# returns JSON object as
# a dictionary
data = json.load(f)

In [ ]:
df=pd.DataFrame(data.items(),columns=['log','anomaly'])
df.head()

,log,anomaly
0,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
1,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
2,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
3,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
4,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal


In [ ]:
df.tail()

,log,anomaly
4152654,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152655,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152656,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152657,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152658,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal


In [37]:
import re
import string
def clean(s):
    """ Preprocess log message
    Parameters
    ----------
    s: str, raw log message
    Returns
    -------
    str, preprocessed log message without number tokens and special characters
    """
    # s = re.sub(r'(\d+\.){3}\d+(:\d+)?', " ", s)
    # s = re.sub(r'(\/.*?\.[\S:]+)', ' ', s)
    s = re.sub('\]|\[|\)|\(|\=|\,|\;', ' ', s)
    s = " ".join([word.lower() if word.isupper() else word for word in s.strip().split()])
    s = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', s))
    s = " ".join([word for word in s.split() if not bool(re.search(r'\d', word))])
    trantab = str.maketrans(dict.fromkeys(list(string.punctuation)))
    content = s.translate(trantab)
    s = " ".join([word.lower().strip() for word in content.strip().split()])
    return s

In [ ]:
df.shape

(4152659, 2)

In [ ]:
df['log'] = df['log'].apply(lambda x: clean(x))

In [ ]:
df.head()

,log,anomaly
0,ras kernel info instruction cache parity error...,normal
1,ras kernel info instruction cache parity error...,normal
2,ras kernel info instruction cache parity error...,normal
3,ras kernel info instruction cache parity error...,normal
4,ras kernel info instruction cache parity error...,normal


In [ ]:
df.log.value_counts()

ras kernel info generating                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       1706751
ras kernel info iar dear                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [ ]:
# df['log'] = df['log'].apply(lambda x: tokenize_and_lemmatize(x))
df.to_csv('Train_Preprocessed.csv', index = False)

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

# Creating an undersampled dataset and more preprocess

In [9]:
df=pd.read_csv('/content/drive/MyDrive/convolve/Train_Preprocessed.csv')

In [10]:
df.head()

,log,anomaly
0,ras kernel info instruction cache parity error...,normal
1,ras kernel info instruction cache parity error...,normal
2,ras kernel info instruction cache parity error...,normal
3,ras kernel info instruction cache parity error...,normal
4,ras kernel info instruction cache parity error...,normal


In [5]:
hmm = df.loc[(df['anomaly'] == 'abnormal')]
hmm.log.value_counts()

ras kernel fatal data tlb error interrupt                                                  67393
ras app fatal ciod failed to read message prefix on control stream cio stream socket to     2164
ras kernel fatal rts panic stopping execution                                                127
ras kernel fatal machine check interrupt                                                       4
ras kernel fatal machine check interrupt bit dcache unit data parity error                     1
ras kernel fatal machine check interrupt bit major internal error                              1
ras kernel fatal ddr machine check register                                                    1
ras kernel fatal machine check interrupt bit dcu read error                                    1
Name: log, dtype: int64

In [6]:
hmm2 = df.loc[(df['anomaly'] == 'normal')]
cnt = hmm2.log.value_counts()
c = 1
sum = 0
for i in cnt.values:
  if i < 20:
    break
  sum = sum + i
  c = c+1

print(sum ,"and ", c) 

4081160 and  514


In [7]:
df2 = pd.read_csv('/content/drive/MyDrive/convolve/Train_Preprocessed.csv')

In [8]:
import random
df3 = df2[df2['anomaly'] == 'normal']
drop_indices = np.random.choice(df3.index, 4013275, replace=False)
df_subset = df3.drop(drop_indices)

In [9]:
df_subset

,log,anomaly
49,ras kernel info instruction cache parity error...,normal
126,ras kernel info instruction cache parity error...,normal
127,ras kernel info instruction cache parity error...,normal
185,ras kernel info instruction cache parity error...,normal
207,ras kernel info instruction cache parity error...,normal
...,...,...
4082807,ras kernel info microseconds spent in the rbs ...,normal
4082816,ras kernel info microseconds spent in the rbs ...,normal
4082870,ras kernel info microseconds spent in the rbs ...,normal
4082923,ras kernel info microseconds spent in the rbs ...,normal


In [10]:
df_subset.log.value_counts()

ras kernel info generating                                                                                                                                                                                                                                                                                                                                                                                    29061
ras kernel info iar dear                                                                                                                                                                                                                                                                                                                                                                                       7451
ras kernel info doublehummer alignment exceptions                                                                                                                                               

In [12]:
df_abnormal = df[df['anomaly'] == 'abnormal']
df_abnormal

,log,anomaly
4082967,ras app fatal ciod failed to read message pref...,abnormal
4082968,ras app fatal ciod failed to read message pref...,abnormal
4082969,ras app fatal ciod failed to read message pref...,abnormal
4082970,ras app fatal ciod failed to read message pref...,abnormal
4082971,ras app fatal ciod failed to read message pref...,abnormal
...,...,...
4152654,ras kernel fatal data tlb error interrupt,abnormal
4152655,ras kernel fatal data tlb error interrupt,abnormal
4152656,ras kernel fatal data tlb error interrupt,abnormal
4152657,ras kernel fatal data tlb error interrupt,abnormal


In [13]:
df_new = pd.concat([df_subset,df_abnormal],ignore_index=True)
df_new

,log,anomaly
0,ras kernel info instruction cache parity error...,normal
1,ras kernel info instruction cache parity error...,normal
2,ras kernel info instruction cache parity error...,normal
3,ras kernel info instruction cache parity error...,normal
4,ras kernel info instruction cache parity error...,normal
...,...,...
139379,ras kernel fatal data tlb error interrupt,abnormal
139380,ras kernel fatal data tlb error interrupt,abnormal
139381,ras kernel fatal data tlb error interrupt,abnormal
139382,ras kernel fatal data tlb error interrupt,abnormal


In [15]:
shuffled = df_new.sample(frac=1,random_state=30,ignore_index=True)
shuffled

,log,anomaly
0,ras kernel info generating,normal
1,ras kernel info generating,normal
2,ras kernel info generating,normal
3,ras kernel fatal data tlb error interrupt,abnormal
4,ras app fatal ciod error loading bglapps swlpe...,normal
...,...,...
139379,ras kernel fatal data tlb error interrupt,abnormal
139380,ras app fatal ciod error loading bglappsfollow...,normal
139381,null null ras mmcs error idoproxydb hit assert...,normal
139382,ras kernel fatal data tlb error interrupt,abnormal


# Counting no.of words and preparing the train data

In [11]:
from collections import Counter
from nltk import ngrams
text_list = [df.iloc[i, 0] for i in range(0, df.shape[0])]
text_full = ' '.join(text_list)

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from string import punctuation
def tokenize_and_lemmatize(string):
  tokens = word_tokenize(string)
  return string


In [14]:
word_count = Counter(ngrams(word_tokenize(text_full), 1))
#bigrams_count = Counter(ngrams(word_tokenize(text_full), 2))

word_count.most_common(10)

[(('ras',), 4051490),
 (('kernel',), 3815050),
 (('info',), 3448712),
 (('generating',), 1706751),
 (('alignment',), 563489),
 (('exceptions',), 563489),
 (('fatal',), 549523),
 (('error',), 514669),
 (('iar',), 457474),
 (('dear',), 442992)]

In [15]:
# Lets only keep words the top 60000 words across all reviews
num_words = len(word_count)
word_list = []
common_words = word_count.most_common(num_words)
for i in range(num_words):
  string = common_words[i][0][0]
  word_list.append(string)

In [18]:
# shuffled.to_csv(r'shuffled.csv')
shuffled = pd.read_csv('/content/drive/MyDrive/convolve/shuffled.csv')

In [19]:
X_train = shuffled['log']
X_train

0                                ras kernel info generating
1                                ras kernel info generating
2                                ras kernel info generating
3                 ras kernel fatal data tlb error interrupt
4         ras app fatal ciod error loading bglapps swlpe...
                                ...                        
139379            ras kernel fatal data tlb error interrupt
139380    ras app fatal ciod error loading bglappsfollow...
139381    null null ras mmcs error idoproxydb hit assert...
139382            ras kernel fatal data tlb error interrupt
139383    ras kernel info ciod missing or invalid fields...
Name: log, Length: 139384, dtype: object

In [ ]:
X_train = X_train.tolist()
X_train

In [21]:
# count = 1
# for i in (df['log'].astype('str')):
#   if 'fatal' in i:
#     print(i, "and ",count)
#   count = count + 1    

In [22]:
def get_sequence(string):
  tokens = word_tokenize(string)
  curr_seq = []
  for token in tokens:
    num = 0
    if token in word_list:
      num = word_list.index(token) + 1
    curr_seq.append(num)
  return curr_seq

In [23]:
X_train_sequence = []
for x in X_train:
  X_train_sequence.append(get_sequence(x))

In [24]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(shuffled['anomaly'])
shuffled['anomaly']=le.transform(shuffled['anomaly'])

In [25]:
shuffled.head()

,Unnamed: 0,log,anomaly
0,0,ras kernel info generating,1
1,1,ras kernel info generating,1
2,2,ras kernel info generating,1
3,3,ras kernel fatal data tlb error interrupt,0
4,4,ras app fatal ciod error loading bglapps swlpe...,1


# Model

In [26]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import sequence
from keras.utils import pad_sequences
max_review_length = 150
X_train_sequence = pad_sequences(X_train_sequence, maxlen=max_review_length)

X_train_sequence = np.array(X_train_sequence)

X_train_sequence.shape


(139384, 150)

In [27]:
y_train = shuffled['anomaly'].tolist()
y_train = np.array(y_train)

In [32]:
embedding_vector_length = 32
modelLSTM = tf.keras.Sequential()

modelLSTM.add(tf.keras.layers.Embedding(
    input_dim = num_words, 
    output_dim = embedding_vector_length,
    input_length = max_review_length
))
for i in range(15):
  modelLSTM.add(tf.keras.layers.LSTM(32, return_sequences = True))
modelLSTM.add(tf.keras.layers.Flatten())
modelLSTM.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

modelLSTM.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])
modelLSTM.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 150, 32)           30048     
                                                                 
 lstm_30 (LSTM)              (None, 150, 32)           8320      
                                                                 
 lstm_31 (LSTM)              (None, 150, 32)           8320      
                                                                 
 lstm_32 (LSTM)              (None, 150, 32)           8320      
                                                                 
 lstm_33 (LSTM)              (None, 150, 32)           8320      
                                                                 
 lstm_34 (LSTM)              (None, 150, 32)           8320      
                                                                 
 lstm_35 (LSTM)              (None, 150, 32)          

In [33]:
history = modelLSTM.fit(X_train_sequence, y_train, batch_size =128,validation_split = 0.3,epochs=10,verbose=1)

Epoch 1/10
763/763 [==============================] - 94s 99ms/step - loss: 0.6932 - accuracy: 0.4995 - val_loss: 0.6931 - val_accuracy: 0.4992
Epoch 2/10
763/763 [==============================] - 71s 94ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5008
Epoch 3/10
763/763 [==============================] - 72s 94ms/step - loss: 0.6932 - accuracy: 0.4966 - val_loss: 0.6931 - val_accuracy: 0.4992
Epoch 4/10
763/763 [==============================] - 71s 94ms/step - loss: 0.6932 - accuracy: 0.4991 - val_loss: 0.6931 - val_accuracy: 0.4992
Epoch 5/10
763/763 [==============================] - 72s 94ms/step - loss: 0.6914 - accuracy: 0.5042 - val_loss: 0.6713 - val_accuracy: 0.5314
Epoch 6/10
763/763 [==============================] - 71s 93ms/step - loss: 0.3062 - accuracy: 0.8290 - val_loss: 0.0380 - val_accuracy: 0.9894
Epoch 7/10
763/763 [==============================] - 71s 92ms/step - loss: 0.0626 - accuracy: 0.9877 - val_loss: 0.0776 - val_accuracy:

In [ ]:
path='/kaggle/working/Weights'
 
# save
modelLSTM.save_weights(path)

In [35]:
test=pd.read_csv('/content/drive/MyDrive/convolve/test.csv')
test.head()

,ID,Log
0,0,1124336301 2005.08.17 R13-M1-N8-C:J12-U01 200...
1,1,1118553175 2005.06.11 R30-M0-N9-C:J16-U01 200...
2,2,1118536033 2005.06.11 R30-M0-N9-C:J16-U01 200...
3,3,1117992566 2005.06.05 R30-M1-N6-C:J03-U11 200...
4,4,1118538965 2005.06.11 R30-M0-N9-C:J16-U01 200...


In [36]:
test.shape

(595300, 2)

In [38]:
import string 
test[' Log'] = test[' Log'].apply(lambda x: clean(x))

In [39]:
X_test=test[' Log']
X_test = X_test.tolist()

In [40]:
from nltk.tokenize import word_tokenize
X_test_sequence = []
for x in X_test:
  X_test_sequence.append(get_sequence(x))

In [43]:
X_test_sequence = pad_sequences(X_test_sequence, maxlen=150)

X_test_sequence = np.array(X_test_sequence)

In [44]:
Y_pred = modelLSTM.predict(X_test_sequence)

18604/18604 [==============================] - 634s 34ms/step


In [45]:
Y_pred

array([[0.99998224],
       [0.00151345],
       [0.00151345],
       ...,
       [0.00151344],
       [0.00151344],
       [0.9998672 ]], dtype=float32)

In [46]:
pred=Y_pred.round()

In [47]:
DF = pd.DataFrame(pred)
DF.reset_index(level=0, inplace=True)
DF=DF.rename(columns={'index':'ID',0:' Label'})
DF[' Label'] = DF[' Label'].astype(str)
DF[' Label'] = DF[' Label'].replace('1.0','normal')
DF[' Label'] = DF[' Label'].replace('0.0','abnormal')
DF



,id,Label
0,0,normal
1,1,abnormal
2,2,abnormal
3,3,normal
4,4,abnormal
...,...,...
595295,595295,normal
595296,595296,normal
595297,595297,abnormal
595298,595298,abnormal


In [48]:
DF=DF.rename(columns={'index':'ID'})
DF

,id,Label
0,0,normal
1,1,abnormal
2,2,abnormal
3,3,normal
4,4,abnormal
...,...,...
595295,595295,normal
595296,595296,normal
595297,595297,abnormal
595298,595298,abnormal


In [49]:
DF.to_csv("submission.csv",index=False)